In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise


# Initialize the video capture object
cap=cv2.VideoCapture(0)
i=0
_,frame=cap.read()
back=None

# Select a Region of Interest (ROI) for the hand
roi=cv2.selectROI(frame)
(x,y,w,h)=tuple(map(int,roi))
while True:
    # Read a frame from the video capture
    _,frame=cap.read()
    if i<60:
        # Update the background model for the first 60 frames
        i+=1
        if back is None:
            back=frame[y:y+h,x:x+w].copy()
            back=np.float32(back)
        else:
            
            cv2.accumulateWeighted(frame[y:y+h,x:x+w].copy(),back,0.2)
    else:
        
        # Convert the background to absolute values and grayscale
        back=cv2.convertScaleAbs(back)
        back_gray=cv2.cvtColor(back,cv2.COLOR_BGR2GRAY)
        frame_gray=cv2.cvtColor(frame[y:y+h,x:x+w],cv2.COLOR_BGR2GRAY)
        
        # Calculate the absolute difference between the background and the current frame
        img=cv2.absdiff(back_gray,frame_gray)

        # Apply thresholding to create a binary mask
        _,img=cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

        # Find contours in the binary mask
        con,hie=cv2.findContours(img,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        img2=img.copy()
        
        # Find the largest contour (hand)
        con=max(con,key=cv2.contourArea)
        conv_hull=cv2.convexHull(con)
        cv2.drawContours(img,[conv_hull],-1,225,3)
        
        # Find the center of the hand
        top=tuple(conv_hull[conv_hull[:,:,1].argmin()][0])
        bottom=tuple(conv_hull[conv_hull[:,:,1].argmax()][0])
        left=tuple(conv_hull[conv_hull[:,:,0].argmin()][0])
        right=tuple(conv_hull[conv_hull[:,:,0].argmax()][0])
        cx=(left[0]+right[0])//2
        cy=(top[1]+bottom[1])//2

        # Estimate the hand radius and create a circular region of interest (ROI)
        dist=pairwise.euclidean_distances([left,right,bottom,top],[[cx,cy]])[0]
        radi=int(0.80*dist)
        circular_roi=np.zeros_like(img,dtype='uint8')
        cv2.circle(circular_roi,(cx,cy),radi,255,8)

        # Apply weighted average to get the background
        wighted=cv2.addWeighted(img.copy(),0.6,circular_roi,0.4,2)

        # Create a mask using the circular ROI
        mask=cv2.bitwise_and(img2,img2,mask=circular_roi)

         # Find contours in the masked region for finger counting
        con,hie=cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
        count=0
        circumfrence=2*np.pi*radi

        # Count fingers based on wrist range and contour length
        for cnt in con:
            (m_x,m_y,m_w,m_h)=cv2.boundingRect(cnt)
            out_wrist_range=(cy+(cy*0.25))>(m_y+m_h)
            limit_pts=(circumfrence*0.25)>cnt.shape[0]
            if limit_pts and out_wrist_range:
                count+=1


        # Display count on the frame 
        cv2.putText(frame,'count: '+str(count),(460,70),cv2.FONT_HERSHEY_SIMPLEX ,1,(0,250,0),thickness=4)
        # Draw rectangles around hand region and display frames
        cv2.rectangle(frame,(x,y),(x+w,y+h),255,3)
        cv2.imshow('mask',mask)
        cv2.imshow('frame',frame)
        cv2.imshow('weight',wighted)

    # Wait for the 'Esc' key to exit the loop    
    k=cv2.waitKey(5)
    if k==27:
        break

# Release resources and close windows    
cap.release()
cv2.destroyAllWindows()

In [3]:
! pip install -U scikit-learn


                                              0.0/9.2 MB ? eta -:--:--
                                              0.0/9.2 MB 660.6 kB/s eta 0:00:14
                                              0.0/9.2 MB 660.6 kB/s eta 0:00:14
                                              0.0/9.2 MB 660.6 kB/s eta 0:00:14
                                              0.1/9.2 MB 218.8 kB/s eta 0:00:42
                                              0.1/9.2 MB 206.9 kB/s eta 0:00:45
                                              0.1/9.2 MB 249.8 kB/s eta 0:00:37
                                              0.1/9.2 MB 327.9 kB/s eta 0:00:28
                                              0.1/9.2 MB 327.9 kB/s eta 0:00:28
                                              0.1/9.2 MB 327.9 kB/s eta 0:00:28
                                              0.1/9.2 MB 327.9 kB/s eta 0:00:28
                                              0.1/9.2 MB 327.9 kB/s eta 0:00:28
     -                                        0.3


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
import numpy as np
from sklearn.metrics import pairwise

# Initialize the video capture object
cap = cv2.VideoCapture(0)
i = 0
_, frame = cap.read()
back = None

# Select a Region of Interest (ROI) for the hand
roi = cv2.selectROI(frame)
(x, y, w, h) = tuple(map(int, roi))
max_radius = int(max(w, h) * 0.8)

while True:
    # Read a frame from the video capture
    _, frame = cap.read()

    if i < 60:
        # Update the background model for the first 60 frames
        i += 1
        if back is None:
            back = frame[y:y + h, x:x + w].copy()
            back = np.float32(back)
        else:
            cv2.accumulateWeighted(frame[y:y + h, x:x + w].copy(), back, 0.2)
    else:
        # Convert the background to absolute values and grayscale
        back = cv2.convertScaleAbs(back)
        back_gray = cv2.cvtColor(back, cv2.COLOR_BGR2GRAY)
        frame_gray = cv2.cvtColor(frame[y:y + h, x:x + w], cv2.COLOR_BGR2GRAY)

        # Calculate the absolute difference between the background and the current frame
        img = cv2.absdiff(back_gray, frame_gray)

        # Apply adaptive thresholding to create a binary mask
        img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)

        # Apply morphological operations to reduce noise
        kernel = np.ones((5, 5), np.uint8)
        img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)

        # Find contours in the binary mask
        con, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        img2 = img.copy()

        # Find the largest contour (hand)
        con = max(con, key=cv2.contourArea)
        conv_hull = cv2.convexHull(con)
        cv2.drawContours(img, [conv_hull], -1, 255, 3)

        # Find the center of the hand
        top = tuple(conv_hull[conv_hull[:, :, 1].argmin()][0])
        bottom = tuple(conv_hull[conv_hull[:, :, 1].argmax()][0])
        left = tuple(conv_hull[conv_hull[:, :, 0].argmin()][0])
        right = tuple(conv_hull[conv_hull[:, :, 0].argmax()][0])
        cx = (left[0] + right[0]) // 2
        cy = (top[1] + bottom[1]) // 2

        # Estimate the hand radius and create a circular region of interest (ROI)
        dist = pairwise.euclidean_distances([left, right, bottom, top], [[cx, cy]])[0]
        radi = int(0.8 * dist)
        circular_roi = np.zeros_like(img, dtype='uint8')
        cv2.circle(circular_roi, (cx, cy), radi, 255, 8)

        # Apply dynamic weighted average to get the background
        weight1 = 0.6
        weight2 = 0.4 * (1 - (radi / max_radius))
        weighted = cv2.addWeighted(img.copy(), weight1, circular_roi, weight2, 2)

        # Create a mask using the circular ROI
        mask = cv2.bitwise_and(img2, img2, mask=circular_roi)

        # Find contours in the masked region for finger counting
       # Find contours in the masked region for finger counting
        con, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        count = sum(1 for cnt in con if ((cy + (cy * 0.25)) > (cnt[:, :, 1].max()) and
                                 (radi * 0.25) > cnt.shape[0]))

        # Display count on the frame
        cv2.putText(frame, 'count: ' + str(count), (460, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 250, 0), thickness=4)

        # Draw rectangles around hand region and display frames
        cv2.rectangle(frame, (x, y), (x + w, y + h), 255, 3)
        cv2.imshow('mask', mask)
        cv2.imshow('frame', frame)
        cv2.imshow('weighted', weighted)

    # Wait for the 'Esc' key to exit the loop
    k = cv2.waitKey(5)
    if k == 27:
        break

# Release resources and close windows
cap.release()
cv2.destroyAllWindows()
